In [1]:
from torch.utils.data import Dataset
import numpy as np
import os

import sqlite3
import torch
import io
import os

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device", device)

def adapt_array(arr):
    out = io.BytesIO()
    np.save(out, arr)
    out.seek(0)
    return sqlite3.Binary(out.read())

def convert_array(text):
    out = io.BytesIO(text)
    out.seek(0)
    return np.load(out)

def position_reformed(data, startgoal):
    datax = (data[0]-startgoal[0])*(3/2) + startx
    datay = (data[1]-startgoal[1])*(3/2) + starty
    return datax, datay

# Converts np.array to TEXT when inserting
sqlite3.register_adapter(np.ndarray, adapt_array)
# Converts TEXT to np.array when selecting
sqlite3.register_converter("array", convert_array)

class TrafficDataset(Dataset):
    def __init__(self, dbpath, train=True, ratio_test=0.8, num_sce=100, num_data=25):
        self.con = sqlite3.connect(dbpath, detect_types=sqlite3.PARSE_DECLTYPES)
        self.path = dbpath
        self.cur = self.con.cursor()
        self.cur.execute("select id from highway")
        self.idlist = self.cur.fetchall()
        self.cur.execute("select data from highway where id = " +  str(1))
        self.dataperow = len(self.cur.fetchone()[0])
        numTrain = int(ratio_test * len(self.idlist))
        if (train):
            self.filelist = self.idlist[:numTrain]
        else:
            self.filelist = self.idlist[numTrain:]
            
    def __len__(self):
        return len(self.idlist) * self.dataperow
    
    def __getitem__(self, idx):
        rowid = idx // self.dataperow
        dataid = idx % self.dataperow
        self.cur.execute("select id, startgoal, occ, data from highway where id = " +  str(rowid+1))
        results = self.cur.fetchone()
        start_goal = results[1].astype(np.single)
        observed = results[2].astype(np.single)
        data = results[3][dataid].astype(np.single)
        
        start_goal[4] = start_goal[4] - start_goal[0]
        start_goal[5] = start_goal[5] - start_goal[1]
        
        data[0] = data[0] - start_goal[0]
        data[1] = data[1] - start_goal[1]
        
        start_goal[0] = 0
        start_goal[1] = 0
        
         
        sample = {'start_goal': start_goal,
                             'observation': observed,
                            'data': data}
        return sample

Using device cuda:0


In [2]:
import torch
import numpy as np

In [3]:
from torch.utils.data import Dataset, DataLoader

dbpath = '/home/rong/disk/database/highway.db'
bs = 8
train_loader = DataLoader(TrafficDataset(dbpath = dbpath,
                            train = True),
                         batch_size = bs, shuffle=True, drop_last = True)
test_loader = DataLoader(TrafficDataset(dbpath = dbpath,
                            train = False),
                          batch_size = bs, shuffle=True, drop_last = True)

# batch = next(iter(train_loader))
# batch['data']

In [22]:
import torch.nn.functional as F
from torch import nn, optim

class convVAE(nn.Module):
    def __init__(self, sample_size, cnnout_size, cond_out_size, encoder_layer_sizes, latent_size, decoder_layer_sizes):
        super(convVAE, self).__init__()

        assert type(encoder_layer_sizes) == list
        assert type(latent_size) == int
        assert type(decoder_layer_sizes) == list
        
        self.latent_size = latent_size
        self.condnn = CondNN(sample_size, cnnout_size)
        self.encoder = Encoder(sample_size + cond_out_size, encoder_layer_sizes, latent_size)
        self.decoder = Decoder(latent_size +cond_out_size, decoder_layer_sizes, sample_size)

    def encode(self, x):
        return self.encoder(x)

    def decode(self, x):
        return self.decoder(x)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def forward(self, x, startend, occ):
        c, _= self.condnn(startend, occ)
#         print("x size: ", x.shape)
#         print("c size", c.shape)
        mu, logvar = self.encode(torch.cat((x, c), dim=1))
        z = self.reparameterize(mu, logvar)
        return self.decode(torch.cat((z, c), dim=-1)), mu, logvar
    
    def inference(self, startend, occ, num_viz):
        c, alpha = self.condnn(startend, occ)
        z = torch.randn(num_viz, self.latent_size, device = c.device)
        return self.decode(torch.cat((z, c), dim=-1)), alpha
    
class Encoder(nn.Module):
    def __init__(self, input_size, layer_sizes, latent_size):
        super(Encoder, self).__init__()

        layer_sizes = [input_size] + layer_sizes
        modules = []
        for i, (in_size, out_size) in enumerate(zip(layer_sizes[:-1], layer_sizes[1:])):
            modules.append(nn.Linear(in_size, out_size))
            modules.append(nn.ReLU())
#             modules.append(nn.Dropout(p=0.5))

        self.sequential = nn.Sequential(*modules)
        self.linear_means = nn.Linear(layer_sizes[-1], latent_size)
        self.linear_log_var = nn.Linear(layer_sizes[-1], latent_size)

    def forward(self, x):
        x = self.sequential(x)
        means = self.linear_means(x)
        log_vars = self.linear_log_var(x)
        return means, log_vars


class Decoder(nn.Module):
    def __init__(self, input_size, layer_sizes, sample_size):
        super(Decoder, self).__init__()

        layer_sizes = [input_size] + layer_sizes
        modules = []
        for i, (in_size, out_size) in enumerate(zip(layer_sizes[:-1], layer_sizes[1:])):
            modules.append(nn.Linear(in_size, out_size))
            modules.append(nn.ReLU())
#             modules.append(nn.Dropout(p=0.5))
        modules.append(nn.Linear(layer_sizes[-1], sample_size))

        self.sequential = nn.Sequential(*modules)

    def forward(self, x):
        return self.sequential(x)


class CondNN(nn.Module):
    def __init__(self, sampleSize, outSize, encoder_dim=64, attention_dim=64):
        super(CondNN, self).__init__()
        self.sampleSize = sampleSize
        self.cnn = Conv3d(cnn_out_size)
        self.Attention = Attention(cnn_encode_dim=encoder_dim, condition_dim=sampleSize * 2, attention_dim=attention_dim) # + 3 for position
        self.fc1 = nn.Linear(encoder_dim + sampleSize * 2, outSize)

    def forward(self, startend, occ):
        cnn_encode = self.cnn(occ)
        attention_weighted_encoding, alpha = self.Attention(cnn_encode, startend)
        x = torch.cat((attention_weighted_encoding, startend), dim=-1)
#         print("condnn cated size:", x.shape)
        x = self.fc1(x)
        return x, alpha

class Conv3d(nn.Module):
    def __init__(self, cnn_out_size):
        super(Conv3d, self).__init__()

        self.adap_pool = nn.AdaptiveAvgPool3d((25, 100, 600))
        self.conv_layer1 = self._make_conv_layer(1, 16)
        self.conv_layer2 = self._make_conv_layer(16, 32)
#         self.conv_layer3 = self._make_conv_layer(64, 124)
        self.conv_layer5=nn.Conv3d(32, 64, kernel_size=(1, 3, 3), padding=0)
        
        self.adap_pool2 = nn.AdaptiveAvgPool3d((6, 10, 60))

    def _make_conv_layer(self, in_c, out_c):
        conv_layer = nn.Sequential(
        nn.Conv3d(in_c, out_c, kernel_size=(2, 3, 3), padding=0),
        nn.LeakyReLU(),
        nn.Conv3d(out_c, out_c, kernel_size=(2, 3, 3), padding=1),
        nn.LeakyReLU(),
        nn.MaxPool3d((2, 2, 2)),
        )
        return conv_layer

    def forward(self, x):
        x = self.adap_pool(x)
#         print(x.size())
        x = self.conv_layer1(x)
#         print(x.size())
        x = self.conv_layer2(x)
#         print(x.size())
        x=self.conv_layer5(x)
#         print(x.size())
        x = self.adap_pool2(x)
#         print(x.size())

        return x
    

class Attention(nn.Module):
    def __init__(self, cnn_encode_dim, condition_dim, attention_dim):
        super(Attention, self).__init__()
        self.encoder_att = nn.Linear(cnn_encode_dim + 3, attention_dim) #位置信息
        self.condition_att = nn.Linear(condition_dim, attention_dim)
        self.full_att = nn.Linear(attention_dim, 1)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)
        self.position = self.initPosition()
    
    def initPosition(self):
        x = np.linspace(0, 5, 6, dtype='float32')
        y = np.linspace(0, 9, 10, dtype='float32')
        t = np.linspace(0, 59, 60, dtype='float32')
        xv, yv, tv = np.meshgrid(x,y,t)
        xv, yv, tv = xv.reshape((1,-1, 1)), yv.reshape((1,-1, 1)), tv.reshape((1,-1, 1))
        position = torch.from_numpy(np.concatenate((xv, yv, tv), axis = 2))
        
        return position
        
        
    def forward(self, encoder_out, condition):
        batch_size = encoder_out.size(0)
        encoder_dim = encoder_out.size(1)
#         print("encoder_dim: ", encoder_dim)
        encoder_out = encoder_out.view(batch_size, -1, encoder_dim)  # (batch_size, num_pixels, encoder_dim)
        self.position = self.position.to(encoder_out.device)
        self.position = self.position.expand(batch_size, self.position.shape[1], self.position.shape[2]) #存疑
#         print("encoder_out: ", encoder_out.shape)
        encoder_out_pos = torch.cat((encoder_out, self.position), dim = 2)
#         print("cat encoder_out: ", encoder_out.shape)
        att1 = self.encoder_att(encoder_out_pos)
        att2 = self.condition_att(condition)# 依然不清晰
        att = self.full_att(self.relu(att1 + att2.unsqueeze(1))).squeeze(2)
        alpha = self.softmax(att)
#         print("alpha shape: ", alpha.shape)
        attention_weighted_encoding = (encoder_out * alpha.unsqueeze(2)).sum(dim=1)
            
        return attention_weighted_encoding, alpha

In [23]:
import torch 

X_dim = 4
z_dim = 50
cnn_out_size = 300
cond_out_size = 300

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device", device)

model = convVAE(sample_size = X_dim, 
                  cnnout_size = cnn_out_size, 
                  cond_out_size = cond_out_size, 
                  encoder_layer_sizes = [512,1024,512], 
                  latent_size = z_dim, 
                  decoder_layer_sizes = [512,1024,512]).to(device)
print(model)

Using device cuda:0
convVAE(
  (condnn): CondNN(
    (cnn): Conv3d(
      (adap_pool): AdaptiveAvgPool3d(output_size=(25, 100, 600))
      (conv_layer1): Sequential(
        (0): Conv3d(1, 16, kernel_size=(2, 3, 3), stride=(1, 1, 1))
        (1): LeakyReLU(negative_slope=0.01)
        (2): Conv3d(16, 16, kernel_size=(2, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        (3): LeakyReLU(negative_slope=0.01)
        (4): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
      )
      (conv_layer2): Sequential(
        (0): Conv3d(16, 32, kernel_size=(2, 3, 3), stride=(1, 1, 1))
        (1): LeakyReLU(negative_slope=0.01)
        (2): Conv3d(32, 32, kernel_size=(2, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        (3): LeakyReLU(negative_slope=0.01)
        (4): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
      )
      (conv_layer5): Conv3d(32, 64, kernel_size=(1, 3, 3), stride=(1, 1, 1))
      (adap

In [24]:
def loss_fn(recon_x, x, w, mean, log_var):
    MSE = torch.mean((w.expand_as(x) * (recon_x-x)**2))
    KLD = - 0.002 * torch.mean(torch.sum(1 + log_var - mean.pow(2) - log_var.exp(), 1))
    return MSE + KLD, MSE

optimizer = optim.Adam(model.parameters(), lr=3e-4)

In [25]:
def train(epoch, writer):
    model.train()
    train_loss = 0
    mse_loss = 0
    w = torch.tensor([5, 10, 1, 3], dtype=torch.float).to(device)
    adap_pool = nn.AdaptiveAvgPool3d((25,100, 600))
    
    for batch_idx, batch in enumerate(train_loader):
        startgoal = batch["start_goal"].to(device)
        occ = batch["observation"]
        occ = adap_pool(occ)
        occ = occ.to(device)
        occ = occ.unsqueeze(1)
        data = batch["data"].to(device)

        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data, startgoal, occ)
        loss, mse= loss_fn(recon_batch, data, w, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        mse_loss += mse.item()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item()))
        
            writer.add_scalar('BatchLoss/loss', loss.item(), batch_idx)
            writer.add_scalar('BatchLoss/mse_loss', mse.item(), batch_idx)

    epoch_loss = train_loss * len(data) / len(train_loader.dataset)
    epoch_mse = mse_loss * len(data) / len(train_loader.dataset)
    print('====> Epoch: {} Average loss: {:.7f}'.format(
          epoch, epoch_loss))
    return epoch, epoch_loss, epoch_mse

def test(epoch):
    model.eval()
    test_loss = 0
    mse_loss = 0
    w = torch.tensor([1, 1, 1, 0.5], dtype=torch.float).to(device)
    for batch_idx, batch in enumerate(test_loader):
        startgoal = batch["start_goal"].to(device)
        occ = batch["observation"].to(device)
        occ = occ.unsqueeze(1)
        data = batch["data"].to(device)
        
        recon_batch, mu, logvar = model(sample, startend, occ)
        loss, mse= loss_fn(recon_batch, data, w, mu, logvar)
        test_loss += loss.item()
        mse_loss += mse.item()

    epoch_loss = test_loss * len(data) / len(test_loader.dataset)
    epoch_mse = mse_loss * len(data) / len(test_loader.dataset)
    print('====> Epoch: {} Average test loss: {:.7f}'.format(
          epoch, epoch_loss))
    return epoch, epoch_loss, epoch_mse

In [8]:
epoch = 0
from torch.utils.tensorboard import SummaryWriter

# Writer will output to ./runs/ directory by default
writer = SummaryWriter('runs/highway_attention_01')

In [9]:
for epoch in range(epoch, epoch + 200):
    log_interval = 20
    epoch, epoch_loss, epoch_mse = train(epoch, writer)
#     vis.line(X=torch.ones((1,1)).cpu()*epoch,Y=torch.Tensor([epoch_loss]).unsqueeze(0).cpu(),win=loss_window,update='append',name='loss')
#         vis.line(X=torch.ones((1,1)).cpu()*epoch,Y=torch.Tensor([epoch_mse]).unsqueeze(0).cpu(),win=loss_window,update='append',name='mse_loss')
    writer.add_scalar('Loss/loss', epoch_loss, epoch)
    writer.add_scalar('Loss/mse_loss', epoch_mse, epoch)

Train Epoch: 0 [0/482500 (0%)]	Loss: 8055.420410
Train Epoch: 0 [160/482500 (0%)]	Loss: 2803.440674
Train Epoch: 0 [320/482500 (0%)]	Loss: 1127.901733
Train Epoch: 0 [480/482500 (0%)]	Loss: 899.661133
Train Epoch: 0 [640/482500 (0%)]	Loss: 1501.718140
Train Epoch: 0 [800/482500 (0%)]	Loss: 228.068207
Train Epoch: 0 [960/482500 (0%)]	Loss: 91.873367
Train Epoch: 0 [1120/482500 (0%)]	Loss: 15.596081
Train Epoch: 0 [1280/482500 (0%)]	Loss: 16.236187
Train Epoch: 0 [1440/482500 (0%)]	Loss: 11.649496
Train Epoch: 0 [1600/482500 (0%)]	Loss: 10.139577
Train Epoch: 0 [1760/482500 (0%)]	Loss: 12.778826
Train Epoch: 0 [1920/482500 (0%)]	Loss: 4.730198
Train Epoch: 0 [2080/482500 (0%)]	Loss: 7.960318
Train Epoch: 0 [2240/482500 (0%)]	Loss: 3.263066
Train Epoch: 0 [2400/482500 (0%)]	Loss: 472.378815
Train Epoch: 0 [2560/482500 (1%)]	Loss: 55.074020
Train Epoch: 0 [2720/482500 (1%)]	Loss: 105.411690
Train Epoch: 0 [2880/482500 (1%)]	Loss: 21.365833
Train Epoch: 0 [3040/482500 (1%)]	Loss: 16.737120


KeyboardInterrupt: 

In [11]:
torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()
            }, 'checkpoints/highway_attention.pt')

In [26]:
torch.cuda.empty_cache()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device", device)

model = convVAE(sample_size = X_dim, 
                  cnnout_size = cnn_out_size, 
                  cond_out_size = cond_out_size, 
                  encoder_layer_sizes = [512,1024,512], 
                  latent_size = z_dim, 
                  decoder_layer_sizes = [512,1024,512]).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

checkpoint = torch.load('checkpoints/highway_attention.pt')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']

model.eval()
print(model)

Using device cuda:0
convVAE(
  (condnn): CondNN(
    (cnn): Conv3d(
      (adap_pool): AdaptiveAvgPool3d(output_size=(25, 100, 600))
      (conv_layer1): Sequential(
        (0): Conv3d(1, 16, kernel_size=(2, 3, 3), stride=(1, 1, 1))
        (1): LeakyReLU(negative_slope=0.01)
        (2): Conv3d(16, 16, kernel_size=(2, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        (3): LeakyReLU(negative_slope=0.01)
        (4): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
      )
      (conv_layer2): Sequential(
        (0): Conv3d(16, 32, kernel_size=(2, 3, 3), stride=(1, 1, 1))
        (1): LeakyReLU(negative_slope=0.01)
        (2): Conv3d(32, 32, kernel_size=(2, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        (3): LeakyReLU(negative_slope=0.01)
        (4): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
      )
      (conv_layer5): Conv3d(32, 64, kernel_size=(1, 3, 3), stride=(1, 1, 1))
      (adap

In [73]:
%matplotlib
from utils.HighWay import plotData, plotOrientSpeed, plotAlpha

Using matplotlib backend: Qt5Agg


In [74]:
test_data = test_loader.dataset
viz_idx =   torch.randint(0,len(test_data),[1]).item()  
#  变道场景idx
#  
print(viz_idx)

batch = test_data[viz_idx]
startgoal = torch.from_numpy(batch["start_goal"]).to(device)
occ = torch.from_numpy(batch["observation"])
occ = occ.unsqueeze(0)
occ = occ.unsqueeze(1)
adap_pool = nn.AdaptiveAvgPool3d((25,100, 600))
occ = adap_pool(occ)
occ = occ.to(device)
data = torch.from_numpy(batch["data"]).to(device)

occ=occ.cpu().detach().numpy()
startgoal=startgoal.cpu().detach().numpy()
data=data.cpu().detach().numpy()
torch.cuda.empty_cache()

plotData(occ, startgoal, data)

358222


In [75]:
test_data = test_loader.dataset
viz_idx =   torch.randint(0,len(test_data),[1]).item()  
#  变道场景idx
#  308958 82146 161608
viz_idx = 161608
print(viz_idx)

batch = test_data[viz_idx]
startgoal = torch.from_numpy(batch["start_goal"]).to(device)
occ = torch.from_numpy(batch["observation"])
occ = occ.unsqueeze(0)
occ = occ.unsqueeze(1)
adap_pool = nn.AdaptiveAvgPool3d((25,100, 600))
occ = adap_pool(occ)
occ = occ.to(device)

data = torch.from_numpy(batch["data"]).to(device)

with torch.no_grad():
    model.eval()
    y_viz = torch.randn(1,4).to(device)
    for i in range(0, 10):
        num_viz = 8
        y_viz_p, alpha = model.inference(startgoal.expand(num_viz, X_dim * 2).to(device), 
                                occ.expand(num_viz, 1, -1, -1, -1).to(device), num_viz)
        torch.cuda.empty_cache()
        y_viz = torch.cat((y_viz_p, y_viz), dim = 0)

y_viz=y_viz.cpu().detach().numpy()
occ=occ.cpu().detach().numpy()
startgoal=startgoal.cpu().detach().numpy()
data=data.cpu().detach().numpy()
alpha=alpha.cpu().detach().numpy()
torch.cuda.empty_cache()
# from utils.NarrowPassage import plotCondition, plotSample, plotSpeed, plotSampleAttention

y_viz=y_viz[:-1]
plotData(occ, startgoal, y_viz)
plotOrientSpeed(startgoal, y_viz)
plotAlpha(alpha)

161608
